# 16. 순환 신경망으로 순차 데이터 모델링

## 16.1 순차 데이터 소개

## 16.2 시퀀스 모델링을 위한 RNN

### 16.2.3 은닉 순환과 출력 순환

In [1]:
import tensorflow as tf
tf.random.set_seed(1)
rnn_layer = tf.keras.layers.SimpleRNN(
    units=2, use_bias=True,
    return_sequences=True)
rnn_layer.build(input_shape=(None, None, 5))
w_xh, w_oo, b_h = rnn_layer.weights
print('W_xh 크기: ', w_xh.shape)
print('W_oo 크기: ', w_oo.shape)
print('b_h 크기: ', b_h.shape)

W_xh 크기:  (5, 2)
W_oo 크기:  (2, 2)
b_h 크기:  (2,)


2022-04-28 11:03:19.641008: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 수동으로 구현하기
x_seq = tf.convert_to_tensor(
    [[1.0]*5, [2.0]*5, [3.0]*5], dtype=tf.float32)
## simple RNN의 출력
output = rnn_layer(tf.reshape(x_seq, shape=(1, 3, 5)))
## 수동으로 출력 계산하기
out_man = []
for t in range(len(x_seq)):
    xt = tf.reshape(x_seq[t], (1, 5))
    print('타임 스텝 {} =>'.format(t))
    print('  입력  : ', xt.numpy())
    
    ht = tf.matmul(xt, w_xh) + b_h
    print('  은닉  : ', ht.numpy())
    
    if t > 0:
        prev_o = out_man[t-1]
    else:
        prev_o = tf.zeros(shape=(ht.shape))
    ot = ht + tf.matmul(prev_o, w_oo)
    ot = tf.math.tanh(ot)
    out_man.append(ot)
    print('  출력 (수동)  :', ot.numpy())
    print('  SimpleRNN 출력: '.format(t), output[0][t].numpy())
    print()

타임 스텝 0 =>
  입력  :  [[1. 1. 1. 1. 1.]]
  은닉  :  [[0.41464037 0.96012145]]
  출력 (수동)  : [[0.39240566 0.74433106]]
  SimpleRNN 출력:  [0.39240566 0.74433106]

타임 스텝 1 =>
  입력  :  [[2. 2. 2. 2. 2.]]
  은닉  :  [[0.82928073 1.9202429 ]]
  출력 (수동)  : [[0.80116504 0.9912947 ]]
  SimpleRNN 출력:  [0.80116504 0.9912947 ]

타임 스텝 2 =>
  입력  :  [[3. 3. 3. 3. 3.]]
  은닉  :  [[1.243921  2.8803642]]
  출력 (수동)  : [[0.95468265 0.9993069 ]]
  SimpleRNN 출력:  [0.95468265 0.9993069 ]



## 16.3 텐서플로로 시퀀스 모델링을 위한 RNN 구현

### 16.3.1 첫 번째 프로젝트: IMDb 영화 리뷰의 감성 분석

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [3]:
# 1 단계 - 데이터셋 만들기
target = df.pop('sentiment')
ds_raw = tf.data.Dataset.from_tensor_slices(
    (df.values, target.values))

# 확인
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 0
b'***SPOILER*** Do not read this, if you think about' 0


In [4]:
# 훈련, 검증, 테스트셋 나누기
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(
    50000, reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(25000)
ds_raw_train_valid = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_valid.take(20000)
ds_raw_valid = ds_raw_train_valid.skip(20000)

In [6]:
# 2 단계 - 고유 토큰 (단어) 찾기
from collections import Counter
tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()
for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)
print('어휘 사전 크기: ', len(token_counts))

어휘 사전 크기:  87007


In [7]:
# 3 단계: 코유 토큰을 정수로 인코딩하기
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = 'This is an example!'
print(encoder.encode(example_str))

[232, 9, 270, 1123]


In [8]:
# 3-A: 변환을 위한 함수 정의
def encode(text_tensor, label):
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text, label

# 3-B: 함수를 TF 연산으로 변환
def encode_map_fn(text, label):
    return tf.py_function(encode, inp=[text, label],
                          Tout=(tf.int64, tf.int64))

In [9]:
ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)

# 샘플의 크기 확인하기
tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('시퀀스 길이: ', example[0].shape)

2022-04-28 09:36:00.226223: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


시퀀스 길이:  (24,)
시퀀스 길이:  (179,)
시퀀스 길이:  (262,)
시퀀스 길이:  (535,)
시퀀스 길이:  (130,)


In [10]:
# 일부 데이터 추출하기
ds_subset = ds_train.take(8)
for example in ds_subset:
    print('개별 샘플 크기: ', example[0].shape)

개별 샘플 크기:  (119,)
개별 샘플 크기:  (688,)
개별 샘플 크기:  (308,)
개별 샘플 크기:  (204,)
개별 샘플 크기:  (326,)
개별 샘플 크기:  (240,)
개별 샘플 크기:  (127,)
개별 샘플 크기:  (453,)


In [11]:
# 배치 데이터 만들기
ds_batched = ds_subset.padded_batch(
    4, padded_shapes=([-1], []))
for batch in ds_batched:
    print('배치 차원: ', batch[0].shape)

배치 차원:  (4, 688)
배치 차원:  (4, 453)


In [12]:
# 배치 크기 32로 만들기
train_data = ds_train.padded_batch(32, padded_shapes=([-1], []))
valid_data = ds_valid.padded_batch(32, padded_shapes=([-1], []))
test_data = ds_test.padded_batch(32, padded_shapes=([-1], []))

In [13]:
# 임베딩 층 만들기
from tensorflow.keras.layers import Embedding
model = tf.keras.Sequential()
model.add(Embedding(input_dim=100, output_dim=6, input_length=20, name='embed-layer'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed-layer (Embedding)      (None, 20, 6)             600       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


#### RNN 모델 만들기

In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          32000     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


#### 감성 분석 작업을 위한 RNN 모델 만들기

In [16]:
embedding_dim = 20
vocab_size = len(token_counts) + 2
tf.random.set_seed(1)

In [17]:
# 모델 만들기
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        name='embed-layer'),
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, name='lstm-layer'),
        name='bidir-lstm'),
    
    tf.keras.layers.Dense(64, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])
bi_lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed-layer (Embedding)      (None, None, 20)          1740180   
_________________________________________________________________
bidir-lstm (Bidirectional)   (None, 128)               43520     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________


In [18]:
# 컴파일 및 훈련
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy'])

history = bi_lstm_model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
625/625 [==============================] - 272s 431ms/step - loss: 0.5908 - accuracy: 0.6500 - val_loss: 0.3444 - val_accuracy: 0.8604
Epoch 2/10
625/625 [==============================] - 250s 400ms/step - loss: 0.2502 - accuracy: 0.9057 - val_loss: 0.3415 - val_accuracy: 0.8698
Epoch 3/10
625/625 [==============================] - 272s 436ms/step - loss: 0.1927 - accuracy: 0.9381 - val_loss: 0.3890 - val_accuracy: 0.8686
Epoch 4/10
625/625 [==============================] - 281s 449ms/step - loss: 0.0979 - accuracy: 0.9674 - val_loss: 0.5154 - val_accuracy: 0.8640
Epoch 5/10
625/625 [==============================] - 276s 442ms/step - loss: 0.0521 - accuracy: 0.9836 - val_loss: 0.5445 - val_accuracy: 0.8480
Epoch 6/10
625/625 [==============================] - 279s 446ms/step - loss: 0.0631 - accuracy: 0.9785 - val_loss: 0.7138 - val_accuracy: 0.8590
Epoch 7/10
625/625 [==============================] - 242s 387ms/step - loss: 0.0710 - accuracy: 0.9757 - val_loss: 0.7107 -

In [19]:
# 평가
test_results = bi_lstm_model.evaluate(test_data)
print('테스트 정확도: {:.2f}%'.format(test_results[1]*100))

782/782 [==============================] - 68s 87ms/step - loss: 0.6807 - accuracy: 0.8457
테스트 정확도: 84.57%


In [5]:
# 전처리 및 토큰 - 텍스트 인코딩 - 배치 데이터 만들기
from collections import Counter
def preprocess_datasets(ds_raw_train, ds_raw_valid, ds_raw_test,
                        max_seq_length=None, batch_size=32):
    # 1단계: (데이터셋 만들기) - 위에서 완료
    # 2단계: 고유 토큰 찾기
    tokenizer = tfds.deprecated.text.Tokenizer()
    token_counts = Counter()
    
    for example in ds_raw_train:
        tokens = tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens = tokens[-max_seq_length:]
        token_counts.update(tokens)
    
    print('어휘 사전 크기: ', len(token_counts))
    
    # 3단계: 텍스트 인코딩하기
    encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
    
    def encode(text_tensor, label):
        text = text_tensor.numpy()[0]
        encoded_text = encoder.encode(text)
        if max_seq_length is not None:
            encoded_text = encoded_text[-max_seq_length:]
        return encoded_text, label
    
    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))
    
    ds_train = ds_raw_train.map(encode_map_fn)
    ds_valid = ds_raw_valid.map(encode_map_fn)
    ds_test = ds_raw_test.map(encode_map_fn)
    
    # 4단계: 배치 데이터 만들기
    train_data = ds_train.padded_batch(batch_size, padded_shape=([-1], []))
    valid_data = ds_valid.padded_batch(batch_size, padded_shape=([-1], []))
    test_data = ds_test.padded_batch(batch_size, padded_shape=([-1], []))
    
    return (train_data, valid_data, test_data, len(token_counts))

In [6]:
# 여러 가지 구조의 모델을 더 쉽게 만들기 위한 헬퍼 함수 만들기
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

def build_rnn_model(embedding_dim, vocab_size,
                    recurrent_type='SimpleRNN',
                    n_recurrent_units=64,
                    n_recurrent_layers=1,
                    bidirectional=True):
    tf.random.set_seed(1)
    
    # 모델 생성
    model = tf.keras.Sequential()
    
    model.add(
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            name='embed-layer')
    )
    
    for i in range(n_recurrent_layers):
        return_sequences = (i < n_recurrent_layers - 1)
        
        if recurrent_type == 'SimpleRNN':
            recurrent_layer = SimpleRNN(
                units=n_recurrent_units,
                return_sequences=return_sequences,
                name='simplernn-layer-{}'.format(i))
        elif recurrent_type == 'LSTM':
            recurrent_layer = LSTM(
                units=n_recurrent_layers,
                return_sequences=return_sequences,
                name='lstm-layer-{}'.format(i))
        elif recurrent_type == 'GRU':
            recurrent_layer = GRU(
                units=n_recurrent_layers,
                return_sequences=return_sequences,
                name='gru-layer-{}'.format(i))
            
        if bidirectional:
            recurrent_layer = Bidirectional(
                recurrent_layer, name='bidir-' + recurrent_layer.name)
        
        model.add(recurrent_layer)
    
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model